In [2]:
import pandas as pd 
import numpy as np 
import glob
import time

import os 
import matplotlib.pyplot as plt 
import keras

import cv2



c:\Users\ADMIN\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.30.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\ADMIN\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.30.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\ADMIN\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.30.1 at tensorflow/core/framewo

In [3]:
df = pd.read_csv('./dataset/XSS_dataset.csv', encoding='utf-8-sig', index_col=0)
df.head(100)


,Sentence,Label
0,"<li><a href=""/wiki/File:Socrates.png"" class=""i...",0
1,"<tt onmouseover=""alert(1)"">test</tt>",1
2,"\t </span> <span class=""reference-text"">Steeri...",0
3,"\t </span> <span class=""reference-text""><cite ...",0
4,"\t </span>. <a href=""/wiki/Digital_object_iden...",0
...,...,...
95,"<li><a href=""/wiki/Mind"" title=""Mind"">Mind </a...",0
96,<spacer id=x tabindex=1 onbeforeactivate=alert...,1
97,<style>@keyframes slidein {}</style><shadow st...,1
98,<noscript id=x tabindex=1 onfocusin=alert(1)><...,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13686 entries, 0 to 13685
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  13686 non-null  object
 1   Label     13686 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 320.8+ KB


In [5]:
# Get sentences data from dataframe
sentences = df['Sentence'].values
sentences[0:10]

array(['<li><a href="/wiki/File:Socrates.png" class="image"><img alt="Socrates.png" src="//upload.wikimedia.org/wikipedia/commons/thumb/c/cd/Socrates.png/18px-Socrates.png" decoding="async" width="18" height="28" class="noviewer" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/c/cd/Socrates.png/27px-Socrates.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/c/cd/Socrates.png/36px-Socrates.png 2x" data-file-width="326" data-file-height="500" /> </a> <a href="/wiki/Portal:Philosophy" title="Portal:Philosophy">Philosophy&#32;portal </a> </li> </ul>',
       '<tt onmouseover="alert(1)">test</tt>',
       '\t </span> <span class="reference-text">Steering for the 1995 "<a href="/wiki/History_of_autonomous_cars#1990s" class="mw-redirect" title="History of autonomous cars">No Hands Across America </a>" required "only a few human assists". ',
       '\t </span> <span class="reference-text"><cite class="citation web"><a rel="nofollow" class="external text" href="https://www.milesed

In [6]:
print(len(sentences))

13686


In [7]:
# Convert to ASCII

def convert_to_ascii(sentence):
    sentence_ascii=[]

    for i in sentence:
        
        """Some characters have values very big e.d 8221 adn some are chinese letters
        I am removing letters having values greater than 8222 and for rest greater 
        than 128 and smaller than 8222 assigning them values so they can easily be normalized"""
       
        if(ord(i)<8222):      # ” has ASCII of 8221
            
            if(ord(i)==8217): # ’  :  8217
                sentence_ascii.append(134)
            
            
            if(ord(i)==8221): # ”  :  8221
                sentence_ascii.append(129)
                
            if(ord(i)==8220): # “  :  8220
                sentence_ascii.append(130)
                
                
            if(ord(i)==8216): # ‘  :  8216
                sentence_ascii.append(131)
                
            if(ord(i)==8217): # ’  :  8217
                sentence_ascii.append(132)
            
            if(ord(i)==8211): # –  :  8211
                sentence_ascii.append(133)
                
                
            """
            If values less than 128 store them else discard them
            """
            if (ord(i)<=128):
                    sentence_ascii.append(ord(i))
    
            else:
                    pass
            

    zer=np.zeros((10000))

    for i in range(len(sentence_ascii)):
        zer[i]=sentence_ascii[i]

    zer.shape=(100, 100)

    return zer

In [8]:
# Send each sentence to be converted to ASCII

arr = np.zeros((len(sentences), 100, 100))

for i in range(len(sentences)): 
    
    image = convert_to_ascii(sentences[i])
    
    x = np.asarray(image, dtype='float')
    image = cv2.resize(x, dsize=(100, 100), interpolation=cv2.INTER_CUBIC)
    image /= 128
    arr[i] = image

In [9]:
print("Input data shape: ", arr.shape)


Input data shape:  (13686, 100, 100)


In [10]:
# Reshape data for input co CNN 
data = arr.reshape(arr.shape[0], 100, 100, 1)

In [11]:
data.shape

(13686, 100, 100, 1)

In [12]:
y = df['Label'].values

In [13]:
# Split into train and test data
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(data, y, test_size=0.2, random_state=42)


In [14]:
# Import libraries for Making model
import tensorflow as tf 
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout, MaxPool2D, BatchNormalization

In [15]:
# A basic CNN Model
# Number of layers = 11 
# Number of Convolutional layer: 3
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation=tf.nn.relu, input_shape=(100, 100, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2), 
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

c:\Users\ADMIN\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [16]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 98, 98, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 49, 49, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 47, 47, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 23, 23, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 21, 21, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 10, 10, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     6,553,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,964,737 (26.57 MB)

 Trainable params: 6,964,737 (26.57 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
class myCallback(tf.keras.callbacks.Callback): 
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy') > 0.97): 
            print("\n Reached 97% + validation accuracy")
            self.model.stop_training=True

callbacks = myCallback()

In [18]:
batch_size = 128
num_epoch = 10
# Model training
model_log = model.fit(X_train, Y_train, batch_size=batch_size, epochs=num_epoch, verbose=1, validation_data=(X_test, Y_test))


Epoch 1/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 110s 1s/step - accuracy: 0.7798 - loss: 0.4629 - val_accuracy: 0.8115 - val_loss: 0.3821
Epoch 2/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 113s 1s/step - accuracy: 0.9082 - loss: 0.2465 - val_accuracy: 0.9675 - val_loss: 0.1136
Epoch 3/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 112s 1s/step - accuracy: 0.9787 - loss: 0.0730 - val_accuracy: 0.9748 - val_loss: 0.0874
Epoch 4/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 101s 1s/step - accuracy: 0.9812 - loss: 0.0642 - val_accuracy: 0.9854 - val_loss: 0.0578
Epoch 5/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 98s 1s/step - accuracy: 0.9858 - loss: 0.0528 - val_accuracy: 0.9836 - val_loss: 0.0553
Epoch 6/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 112s 1s/step - accuracy: 0.9878 - loss: 0.0452 - val_accuracy: 0.9854 - val_loss: 0.0479
Epoch 7/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 116s 1s/step - accuracy: 0.9890 - loss: 0.0386 - val_accuracy: 0.9858 - val_loss: 0.0487
Epoch 8/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 111s 1s/step - accuracy: 0.9905 - loss: 0.0336 - val_accuracy: 0.9887 - va

In [19]:
pred = model.predict(X_test)

86/86 ━━━━━━━━━━━━━━━━━━━━ 11s 120ms/step


In [20]:
# Threshold values predicted

for i in range(len(pred)): 
    if pred[i] > 0.5: 
        pred[i] = 1
    elif pred[i] <= 0.5:
        pred[i] = 0

In [21]:
# Count True predicted and wrong predicted
true = 0
false = 0

for i in range(len(pred)): 
    if pred[i] == Y_test[i]: 
        true += 1
    else: 
        false += 1

print("Correct predicted : ", true)
print("false prediction: ", false)

Correct predicted :  2702
false prediction:  36


In [24]:
# Number of attack and benign data in test set
attack = 0
benign = 0
for i in range(len(Y_test)): 
    if Y_test[i] == 1: 
        attack += 1
    else:
        benign += 1

print("Attack data in test set: ", attack)
print("Benign data in test set: ", benign)


Attack data in test set:  1478
Benign data in test set:  1260


## Confusion matrix

In [25]:
# Take True positive, true negative, false positive and false negative as input
# Return Accuracy

def accuracy_function(tp, tn, fp, fn): 
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    
    return accuracy

In [26]:
# Take True Positive and false positive
# Return Precision
def precision_function(tp, fp): 
    precision = tp / (tp + fp)
    
    return precision

In [33]:
# Take True Positive and false negative
# Return Recall
def recall_function(tp, fn): 
    recall = tp / (tp + fn)
    
    return recall

In [34]:
#Takes true values and predicted
# Returns Accuracy, precision and recall


def confusion_matrix(truth,predicted):
    
    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0
    
    for true,pred in zip(truth,predicted):
        if true == 1:
            if pred == true:
                true_positive += 1
            elif pred != true:
                false_negative += 1

        elif true == 0:
            if pred == true:
                true_negative += 1
            elif pred != true:
                false_positive += 1
            
    accuracy=accuracy_function(true_positive, true_negative, false_positive, false_negative)
    precision=precision_function(true_positive, false_positive)
    recall=recall_function(true_positive, false_negative)
    
    return (accuracy, precision, recall)

In [35]:
accuracy, precision, recall = confusion_matrix(Y_test, pred)


In [36]:
print("Accuracy: {0} \n Precision: {1} \n Recall: {2}".format(accuracy, precision, recall))

Accuracy: 0.9868517165814463 
 Precision: 0.989145183175034 
 Recall: 0.986468200270636
